In [4]:
static = spark.read.json("./data/activity-data/")
dataSchema = static.schema

In [6]:
print(dataSchema)

StructType(List(StructField(Arrival_Time,LongType,true),StructField(Creation_Time,LongType,true),StructField(Device,StringType,true),StructField(Index,LongType,true),StructField(Model,StringType,true),StructField(User,StringType,true),StructField(gt,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(z,DoubleType,true)))


In [7]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json("./data/activity-data")

In [8]:
activityCounts = streaming.groupBy("gt").count()

In [9]:
# 로컬 머신에서 작동시키기 위해 파티션 줄이기
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [10]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start()

21/10/10 16:15:56 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/50/34qx4w51577d2hlff47g2fp00000gn/T/temporary-0b0c316a-40f6-46fd-ac1f-35855c4433e8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [11]:
activityQuery.awaitTermination()

KeyboardInterrupt: 

In [13]:
spark.streams.active

In [12]:
# 1초마다 결과 출력
from time import sleep

for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       s

In [14]:
from pyspark.sql.functions import expr

simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time")\
.writeStream\
.queryName("simple_transform")\
.format("memory")\
.outputMode("append")\
.start()

21/10/10 16:27:30 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/50/34qx4w51577d2hlff47g2fp00000gn/T/temporary-2359ace8-7557-4b0a-9251-696a5de85963. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [15]:
deviceModelStats = streaming.cube("gt", "model").avg()\
.drop("avg(Arrival_time)")\
.drop("avg(Creation_time)")\
.drop("avg(Index)")\
.writeStream.queryName("device_counts").format("memory")\
.outputMode("complete")\
.start()

21/10/10 16:28:51 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/50/34qx4w51577d2hlff47g2fp00000gn/T/temporary-72aad729-93a2-4963-a42d-85ce66cba53b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [18]:
SELECT * FROM device_counts

SyntaxError: invalid syntax (2728417955.py, line 1)

In [19]:
historicalAgg = static.groupBy("gt", "model").avg()
deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")\
.cube("gt", "model").avg()\
.join(historicalAgg, ["gt", "model"])\
.writeStream.queryName("device_counts").format("memory")\
.outputMode("complete")\
.start()

21/10/10 16:32:29 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/50/34qx4w51577d2hlff47g2fp00000gn/T/temporary-500432a9-b827-4a44-83db-f7bfc334da4a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


IllegalArgumentException: Cannot start query with name device_counts as a query with that name is already active in this SparkSession

In [20]:
# topic1 수신
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "host1:port1, host2:port2")\
.option("subscribe", "topic1")\
.load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".

In [21]:
# 여러개 토픽 수신
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "host1:port1, host2:port2")\
.option("subscribe", "topic1, topic2")\
.load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".

In [22]:
# 패턴에 맞는 토픽 수신
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "host1:port1, host2:port2")\
.option("subscribe", "topic.*")\
.load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".

In [ ]:
df1.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")\
.writeStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "host1:port1, host2:port2")\
.option("checkpointLocation", "/to/HDFS-compatible/dir")\
.start()

In [ ]:
activityCounts.writeStream.trigger(processingTime='5 seconds')\
.format("console").outputMode("complete").start()

In [ ]:
activityCounts.writeStream.trigger(once=True)\
.format("console").outputMode("complete").start()